### Initialise

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format ='retina'

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

ib_extract_file_name = "ib_24122024.csv"
source_folder = "IB_extract"
target_folder = "current_data"

"""
Function to Extract respective sections from IB extract csv file
"""
def extract(section_name, section_description):
    with open(f'{source_folder}/{ib_extract_file_name}', 'r') as source_file:
        lines = source_file.readlines()

    start_index = None
    end_index = None
    bos = f'"BOS","{section_name}","{section_description}"'
    eos = f'"EOS","{section_name}"'

    for i, line in enumerate(lines):
        if bos in line:
            start_index = i + 1
        if eos in line and start_index is not None:
            end_index = i - 1
            break

    if start_index is not None and end_index is not None:
        with open(f'{source_folder}/temp.csv', 'w') as positions_file:
            for line in lines[start_index:end_index + 1]:
                positions_file.write(line)

### Extract Cash Report (CRTT)

In [ ]:
section_name = "CRTT"
extract(section_name, "Cash Report; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "LevelOfDetail",
        "FromDate", 
        "ToDate", 
        "Commissions",
        "CommissionsYTD",
        "Deposits",
        "DepositsYTD",
        "Withdrawals",
        "WithdrawalsYTD",
        "BrokerInterest",
        "BrokerInterestYTD",
        "NetTradesSales",
        "NetTradesSalesYTD",
        "NetTradesPurchases",
        "NetTradesPurchasesYTD",
        "SalesTax",
        "EndingCash",
        "EndingSettledCash",
        "NetCashBalanceSLB",
        "NetSettledCashBalanceSLB",
    ]

data = pd.read_csv(f'{source_folder}/temp.csv')
cash_report = data[columns].copy()
cash_report.to_csv(f'{target_folder}/{section_name}.csv', index=False)
cash_report.head()

### Extract Cash Transactions (CTRN)

In [ ]:
section_name = "CTRN"
extract(section_name, "Cash Transactions")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "FXRateToBase",
        "Date/Time", 
        "SettleDate", 
        "Amount",
        "Type",
        "TransactionID",
        "ReportDate",
    ]
data = pd.read_csv(f'{source_folder}/temp.csv')
source_ctrn = data[columns].copy()

data = pd.read_csv(f'{target_folder}/{section_name}.csv')
current_ctrn = data[columns].copy()

# Find TransactionIDs that are in source_ctrn but not in current_ctrn
new_trades = source_ctrn[~source_ctrn['TransactionID'].isin(current_ctrn['TransactionID'])]

# Append new transactions to current_ctrn
updated_trades = pd.concat([current_ctrn, new_trades])

# Write the updated dataframe to final.csv
updated_trades.to_csv(f'{target_folder}/{section_name}.csv', index=False)

### Positions (POST)

In [ ]:
section_name = "POST"
extract(section_name, "Position; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "AssetClass",
        "Symbol", 
        "Description", 
        "Conid",
        "SecurityID",
        "ListingExchange",
        "IssuerCountryCode",
        "ReportDate",
        "Quantity",
        "CostBasisPrice",
        "CostBasisMoney",
        "PercentOfNAV",
    ]
data = pd.read_csv(f'{source_folder}/temp.csv')
positions = data[columns].copy()
positions.to_csv(f'{target_folder}/{section_name}.csv', index=False)
positions.head(20)

### Trades (TRNT)

In [ ]:
section_name = "TRNT"
extract(section_name, "Trades; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "AssetClass",
        "Symbol", 
        "Description", 
        "Conid",
        "SecurityID",
        "ListingExchange",
        "IssuerCountryCode",
        "TradeID",
        "TradeDate",
        "SettleDateTarget",
        "TransactionType",
        "Exchange",
        "Quantity",
        "TradePrice",
        "TradeMoney",
        "Taxes",
        "IBCommission",
        "NetCash",
        "CostBasis",
        "FifoPnlRealized",
        "Buy/Sell",
        "LevelOfDetail",
    ]
data = pd.read_csv(f'{source_folder}/temp.csv')
trades = data[columns].copy()

# Fetch current trades snapshot data
data = pd.read_csv(f'{target_folder}/{section_name}.csv')
current_trades = data[columns].copy()

# Find records that are in source trades but not in current_trades snapshot
new_trades = trades[~trades.apply(tuple,1).isin(current_trades.apply(tuple,1))]

# Append new transactions to current_trades
updated_trades = pd.concat([current_trades, new_trades])

# # Write the updated dataframe to final.csv
updated_trades.to_csv(f'{target_folder}/{section_name}.csv', index=False)
updated_trades.head(10)



## Lock Files

In [ ]:
from cryptography.fernet import Fernet

# Create new key - YOU SHOULD NOT NEED THIS ANYMORE
# def write_key():
#     key = Fernet.generate_key()
#     with open("folio.key", "wb") as key_file:
#         key_file.write(key)

# write_key()

#### Encrypt

In [ ]:
from cryptography.fernet import Fernet
import os

folio_key = os.getenv("FOLIO_KEY")

f = Fernet(folio_key)

# Define the directories
current_data_dir = "current_data"
encrypt_dir = "current_data"

# Ensure the encrypt directory exists
os.makedirs(encrypt_dir, exist_ok=True)

# Encrypt each file in the current_data directory
for filename in os.listdir(current_data_dir):
    file_path = os.path.join(current_data_dir, filename)
    
    # Read the file data
    with open(file_path, "rb") as file:
        file_data = file.read()

    # Encrypt data
    decrypted_data = f.encrypt(file_data)

    # Write the encrypted file to the encrypt directory
    decrypted_file_path = os.path.join(encrypt_dir, filename)
    with open(decrypted_file_path, "wb") as file:
        file.write(decrypted_data)

print("Encryption complete. Encrypted files are saved in the 'encrypt' folder.")



#### Decrypt

In [ ]:
folio_key = os.getenv("FOLIO_KEY")

f = Fernet(folio_key)

encrypt_dir = "current_data"
decrypt_dir = "current_data"

# Ensure the encrypt directory exists
os.makedirs(decrypt_dir, exist_ok=True)

# Encrypt each file in the current_data directory
for filename in os.listdir(encrypt_dir):
    file_path = os.path.join(encrypt_dir, filename)
    
    # Read the file data
    with open(file_path, "rb") as file:
        file_data = file.read()

    # Decrypt data
    decrypted_data = f.decrypt(file_data)

    # Write the encrypted file to the encrypt directory
    decrypted_file_path = os.path.join(decrypt_dir, filename)
    with open(decrypted_file_path, "wb") as file:
        file.write(decrypted_data)

print("Decryptiong complete. Decrypted files are saved in the 'decrypt' folder.")